# Library Demo: IPI Analysis Toolkit

## 📚 Overview

This interactive notebook provides a comprehensive guide to the custom Python library developed for the **Idaho Policy Institute (IPI)** municipal finance and crime analysis project.

### Purpose

This toolkit simplifies complex data operations by providing:
- **Automated data loading** from multiple sources (IPI, BLS, FBI, Census Bureau)
- **Feature engineering** with inflation adjustment and normalization
- **Search utilities** to navigate 600+ variables
- **Visualization functions** for publication-ready plots

### Who Should Use This

- **Researchers** analyzing Idaho municipal data
- **Policy analysts** exploring relationships between finances and crime
- **Data scientists** looking for reusable data processing pipelines
- **Students** learning about feature engineering and statistical analysis

---

## 📖 Notebook Contents

1. **Program Structure** - Organization of files and modules
2. **Loading Data Functions** - How to import and merge datasets
3. **Supporting Functions** - Data processing and feature engineering utilities
4. **Visualization Functions** - Creating insightful plots
5. **Best Practices** - Tips for efficient analysis

---

**Author:** Dominik Huffield  
**Project:** Strategic Financial Insight - Idaho Policy Institute  
**Last Updated:** 2019


## 1. Program Structure

Our project follows a modular architecture for maintainability and reusability:

---

### 📓 Jupyter Notebooks

| Notebook | Purpose | Key Features |
|----------|---------|--------------|
| **Library_Demo.ipynb** | Tutorial (this file) | Function documentation, usage examples |
| **Final_Models.ipynb** | Complete analysis pipeline | Data exploration, feature selection, OLS modeling |

---

### 📦 Support Modules (`support/`)

Custom Python modules that power the analysis:

| Module | Purpose | Key Functions |
|--------|---------|---------------|
| **`__init__.py`** | Package initializer | Makes `support` a Python package |
| **`load_data.py`** | Data loading & merging | `all_data()`, `ipi_abb()`, `emp()`, `gps()` |
| **`supporting_funcs.py`** | Feature engineering | `normalize()`, `gen_real_dollars()`, `search_all()` |
| **`plotting_funcs.py`** | Visualization | `plot_year()`, `plot_corr_matrix()`, `plot_scatter_matrix()` |

---

### 📊 Data Files

#### **Primary Datasets**
- **`Idaho_Municipal_Database_03052019.xlsx`** - Complete IPI financial dataset (1995-2014)
- **`ipi_final.csv`** - Fully processed & merged data (generated by `load.all_data()`)
- **`col_only.csv`** - Column metadata with full descriptions

#### **Auxiliary Data**
- **`emp_data.csv`** - BLS employment statistics for Idaho counties
- **`gps_data.csv`** - Geographic coordinates by city/ZIP
- **`bls_cpi_stats.xlsx`** - CPI data for inflation adjustment
- **`best_cities.csv`** - 59 cities with most complete data

#### **Data Folders**
- **`employment/`** - Yearly BLS employment Excel files (1995-2015)
- **`GPS/`** - Geographic reference files for city/ZIP matching

---

### 💡 Quick Start

To begin analysis, simply load the complete dataset:

```python
import support.load_data as load
data = load.all_data(out=True, norm=True)
```

This single command:
1. ✅ Loads IPI financial data
2. ✅ Adjusts for inflation (to Oct 2019 dollars)
3. ✅ Merges employment statistics
4. ✅ Adds geographic coordinates
5. ✅ Creates normalized features
6. ✅ Categorizes cities by size
7. ✅ Saves to `ipi_final.csv` for reuse

---

## 2. Import Libraries

Before using any functions, we need to import the necessary modules.

### Standard Imports

In [ ]:
# Standard data science libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# Custom IPI modules
import support.load_data as load           # Data loading functions
import support.supporting_funcs as funcs   # Feature engineering utilities
import support.plotting_funcs as plotipi   # Visualization functions

# Display settings for better output
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 100)

print("✓ All libraries imported successfully!")

---

# 3. Data Loading Functions

The `support.load_data` module provides functions to load and merge data from various sources.

## Function Overview

| Function | Returns | Use Case |
|----------|---------|----------|
| `all_data()` | Complete merged dataset | Primary function - use for most analyses |
| `ipi_abb()` | Abbreviated dataset | High-quality subset (4 years, 59 cities) |
| `cols()` | Column metadata | Look up full descriptions |
| `emp()` | Employment data | BLS statistics only |
| `gps()` | Geographic data | Coordinates only |

---

## 3.1 Loading Complete Dataset: `all_data()`

This is the **primary function** for loading analysis-ready data. It performs a complete preprocessing pipeline:

### What It Does

1. **Loads IPI Financial Data** - Revenue, expenditure, debt from Idaho municipalities
2. **Inflation Adjustment** - Converts all dollar values to October 2019 dollars using CPI
3. **Merges Employment Data** - BLS county-level employment statistics
4. **Adds Geographic Data** - Latitude/longitude coordinates for mapping
5. **Creates Normalized Features** - Per-capita rates and percentage features
6. **Categorizes Cities** - Rural (<2.5k), Non-urban (2.5k-50k), Urban (>50k)

### Parameters

- **`out`** (bool): If `True`, saves to `ipi_final.csv` for faster future loading
- **`norm`** (bool): If `True`, creates normalized features (recommended)

### Usage Example

Let's load the complete dataset:


In [ ]:
# Load complete dataset with all preprocessing
# This may take 1-2 minutes on first run
ipi_data = load.all_data(out=True, norm=True)

print(f"\n✓ Dataset loaded successfully!")
print(f"  Shape: {ipi_data.shape[0]:,} rows × {ipi_data.shape[1]} columns")
print(f"  Years covered: {ipi_data['Year4'].min()} - {ipi_data['Year4'].max()}")
print(f"  Cities: {ipi_data['Name'].nunique()}")

# TIP: After first run, load from CSV for faster access:
# ipi_data = pd.read_csv("ipi_final.csv")

Loading IPI data
Adjusting for Inflation
Getting GPS
Getting Employees
Merge Everything
Normalize Columns
Categorize City Size
writing file to ipi_final.csv


### Preview the Data

The dataset includes all available cities and years. Let's examine the first few rows:

In [ ]:
# Display first few rows
ipi_data.head()

# Note the mix of original and normalized columns:
# - Original: Total_Expenditure, Total_Revenue, Population
# - Normalized: *_PerExp, *_PerRev, *_100k

,Name,County,Year4,YearofData,SurveyYr,ID,IDChanged,Type_Code,County,FIPS_Code_State,...,Counties - Burglary- Total_100k,Counties - Manslaughter- Total_100k,Counties - Murder- Total_100k,Counties - Rape-Total_100k,Counties - Robbery- Total_100k,N of Registered Organizations_100k,N of Orgs Filing Form 990_100k,Total_Expenditure_100k,Total_Revenue_100k,size
0,BOISE CITY,Ada,1996,96.0,96.0,132001001.0,0.0,2.0,Ada,16.0,...,1723.223580,5.237762,4.583041,51.722896,45.175694,454.375822,145.347886,124946.494324,130437.172830,urban
1,KUNA CITY,Ada,1996,96.0,96.0,132001003.0,0.0,2.0,Ada,16.0,...,93499.111901,284.191829,248.667851,2806.394316,2451.154529,24653.641208,7886.323268,54420.553644,53837.892470,non-urban
2,MERIDIAN CITY,Ada,1996,95.0,96.0,132001004.0,0.0,2.0,Ada,16.0,...,12759.974790,38.784118,33.936103,382.993164,334.513017,3364.522228,1076.259272,52528.739504,59263.804954,non-urban
3,BOISE CITY,Ada,1997,97.0,97.0,132001001.0,0.0,2.0,Ada,16.0,...,1525.498078,0.654720,9.166083,70.709782,48.449295,485.147672,156.478129,149601.991377,138227.629401,urban
4,EAGLE CITY,Ada,1997,97.0,97.0,132001801.0,0.0,2.0,Ada,16.0,...,35426.486240,15.204501,212.863007,1642.086057,1125.133039,11266.534894,3633.875627,27280.016597,29132.814865,non-urban


---

## 3.2 Loading Abbreviated Dataset: `ipi_abb()`

For cleaner analysis with the highest quality data, use the abbreviated dataset.

### Key Differences from `all_data()`

| Feature | `all_data()` | `ipi_abb()` |
|---------|--------------|-------------|
| **Years** | All available (1995-2014) | 4 key years (1997, 2002, 2007, 2012) |
| **Cities** | All cities (~100+) | Top 59 cities with complete data |
| **Data Quality** | Mixed | High - minimal missing values |
| **Use Case** | Exploratory analysis | Statistical modeling |

### Why These Years?

We identified a pattern of consistent data collection every 5 years. These 4 years have:
- ✓ Most complete financial records
- ✓ Consistent variable availability
- ✓ Minimal missing data across cities

### Usage Example

In [ ]:
# Load abbreviated dataset (requires ipi_final.csv to exist)
abb_data = load.ipi_abb()

print(f"\n✓ Abbreviated dataset loaded!")
print(f"  Shape: {abb_data.shape[0]:,} rows × {abb_data.shape[1]} columns")
print(f"  Years: {sorted(abb_data['Year4'].unique())}")
print(f"  Cities: {abb_data['Name'].nunique()}")

# Display sample
abb_data.head()

note this requires ipi_final.csv to be created (call all_data(out=True)


,Name,County,Year4,YearofData,SurveyYr,ID,IDChanged,Type_Code,County.1,FIPS_Code_State,...,Counties - Manslaughter- Total_100k,Counties - Murder- Total_100k,Counties - Rape-Total_100k,Counties - Robbery- Total_100k,N of Registered Organizations_100k,N of Orgs Filing Form 990_100k,Total_Expenditure_100k,Total_Revenue_100k,size,DataCount
0,BOISE CITY,Ada,1997,97.0,97.0,132001001.0,0.0,2.0,Ada,16.0,...,0.654720,9.166083,70.709782,48.449295,485.147672,156.478129,149601.991377,138227.629401,urban,4
1,BOISE CITY,Ada,2002,NaN,2.0,132001001.0,0.0,2.0,Ada,16.0,...,0.538251,3.229505,87.196628,60.284089,509.185250,173.316755,153065.175562,135228.938273,urban,4
2,BOISE CITY,Ada,2007,NaN,7.0,132001001.0,NaN,2.0,Ada,16.0,...,0.503428,9.061710,103.202811,46.818836,611.162013,237.618180,128140.636956,152203.168916,urban,4
3,BOISE CITY,Ada,2012,NaN,12.0,132001001.0,NaN,2.0,Ada,16.0,...,0.000000,0.972427,59.318037,39.383287,645.691420,255.748258,147318.465719,148231.314079,urban,4
4,GARDEN CITY,Ada,1997,NaN,97.0,132001002.0,0.0,2.0,Ada,16.0,...,11.475786,160.661005,1239.384898,849.208171,8503.557494,2742.712876,90290.256736,78293.263177,non-urban,4


---

## 3.3 Understanding Column Names: `cols()`

With 600+ variables, we shortened column names for convenience. Full descriptions are preserved in `col_only.csv`.

### Column Metadata Structure

Each variable has:
- **ShortName** - Concise identifier used in dataset (e.g., `Total_Crime`)
- **LongName** - Full description with source, unit, time period, and notes

### Usage Example

Let's view the column metadata:

In [ ]:
# Load column descriptions
ipi_cols = load.cols()

print(f"Total variables documented: {len(ipi_cols)}")
print("\nSample column descriptions:")

# Show last few entries
ipi_cols.tail()

,Index,LongName,ShortName
613,613,"N of Orgs Filing Form 990: Note: ""Includes org...",N of Orgs Filing Form 990
614,614,Business Establishments: Source: Census Bureau...,Business Establishments
615,615,Population Density // RAND State Statistics =>...,Population Density
616,616,"Personal income per capita // Unit: dollars, c...",Personal income per capita
617,617,Population2: Current Population Estimates,Population2: Current Population Estimates


---

## 3.4 Additional Loading Functions

These functions load individual data sources. They're called internally by `all_data()`, but can be used independently.

### Individual Data Loaders

```python
# Load employment data only
emp_data = load.emp()        # Pre-processed employment file
emp_data = load.empl(out=True)  # Compile from raw Excel files

# Load geographic data only
gps_data = load.gps()        # City coordinates for mapping
```

### When to Use

- **Individual loaders**: When you need only one data source
- **`all_data()`**: For comprehensive analysis (recommended)
- **`ipi_abb()`**: For statistical modeling with clean data

---

# 4. Supporting & Processing Functions

The `support.supporting_funcs` module provides utilities for data exploration and feature engineering.

## Function Overview

| Function | Purpose | Common Use |
|----------|---------|------------|
| `search_all()` | Find columns by pattern | Locate variables for analysis |
| `search_column()` | Search column descriptions | Understand what variables measure |
| `normalize()` | Create per-capita/percentage features | Compare cities of different sizes |
| `categorize_size()` | Classify cities by population | Group analysis by urban/rural |
| `gen_real_dollars()` | Adjust for inflation | Make temporal comparisons |

**Note:** Most preprocessing functions are called automatically by `load.all_data()`. These examples help you understand what's happening under the hood.

---

## 4.1 Finding Variables: `search_all()`

With 600+ columns, finding the right variables can be challenging. `search_all()` uses case-insensitive regex to locate columns by pattern.

### Features
- ✓ Case-insensitive search
- ✓ Regex pattern matching
- ✓ Returns list of matching column names
- ✓ Optionally silent for programmatic use

### Syntax
```python
search_all(data, search_pattern, silent=False)
```

### Example: Find All Revenue Columns

Let's search for columns related to "total revenue":

In [ ]:
# Search for total revenue columns using regex
# Pattern: "total" followed by anything, then "revenue"
revenue_cols = funcs.search_all(ipi_data, 'total.*revenue')

# The function prints matches and returns them as a list
print(f"\nFound {len(revenue_cols)} matching columns")
print("You can now use these in analysis:")
print(f"  df[{revenue_cols[:2]}]  # Select first two matches")

Total_Revenue
Total_IG_Revenue
Total_Fed_IG_Revenue
Total_State_IG_Revenue
Total_Utility_Revenue
Total_IG_Revenue_PerRev
Total_Fed_IG_Revenue_PerRev
Total_State_IG_Revenue_PerRev
Total_Utility_Revenue_PerRev
Total_Revenue_100k


array(['Total_Revenue', 'Total_IG_Revenue', 'Total_Fed_IG_Revenue',
       'Total_State_IG_Revenue', 'Total_Utility_Revenue',
       'Total_IG_Revenue_PerRev', 'Total_Fed_IG_Revenue_PerRev',
       'Total_State_IG_Revenue_PerRev', 'Total_Utility_Revenue_PerRev',
       'Total_Revenue_100k'], dtype=object)

**Notice:** The search found both original columns (e.g., `Total_Revenue`) and normalized versions (e.g., `Total_Revenue_100k`).

### Pro Tips

```python
# Silent mode for programmatic use
crime_vars = funcs.search_all(ipi_data, 'crime', silent=True)

# Simple keyword search
funcs.search_all(ipi_data, 'police')

# Complex regex patterns
funcs.search_all(ipi_data, '^total_.*_100k$')  # Total variables per 100k
```

---

## 4.2 Understanding Variables: `search_column()`

While `search_all()` finds column names, `search_column()` searches the full descriptions to understand what each variable measures.

### When to Use

- Unsure what a column measures?
- Need to find all columns from a specific source (FBI, BLS, Census)?
- Want complete variable documentation?

### Example: Search for Assault Data


In [ ]:
# Search column descriptions for "assault"
# Setting disp=True shows the full LongName descriptions
assault_cols = funcs.search_column('assault', ipi_cols, disp=True)

print(f"\n\nFound {len(assault_cols)} assault-related variables:")
print(assault_cols)

# TIP: Use this to understand:
# - Data source (FBI, Census, etc.)
# - Time period covered
# - Unit of measurement
# - Any data quality notes

Assault, Total - Municipalities // Source: RAND => Crimes, Arrests, & Clearances (Description: "This database reports the number of offenses (murder and nonnegligent manslaughter, forcible rape, robbery, aggravated assault, burglary, larceny-theft, motor vehicle theft, and arson) and auxiliary offense data (e.g., burglary: forcible entry, unlawful entry, and attempted entry) for each agency reporting data to the Uniform Crime Reporting Program (UCR) in U.S. States. See crime definitions for crime descriptions. 
The database also reports the number of reported offenses cleared by arrest or exceptional means and the number of clearances which involved only juveniles (under 18 years of age). The category "total offenses cleared by an arrest" do not contain data for the years 1960-1963."; Originating source: Federal Bureau of Investigation; Unit: municipalities) => Area: Cities in Idaho => Offense: Assault total, Burglary total, Manslaughter, Murder, Rape total, Robbery total, Total all th

array(['Assault- Total - Municipalities', 'Counties - Assault- Total'],
      dtype=object)

---

## 4.3 Feature Engineering: `normalize()`

Raw financial data is difficult to compare across cities of different sizes. A $1M police budget means different things in Boise vs. a small town.

### The Problem

❌ **Raw values aren't comparable:**
- Large cities naturally have higher total expenditures
- Raw crime counts don't account for population
- Can't identify spending priorities from absolute dollars

✅ **Solution: Normalize to comparable metrics**

### Three Types of Normalized Features

The `normalize()` function creates three categories of features:

#### 1. **Expenditure Percentages** (`*_PerExp`)
- **Formula:** (Category Expenditure / Total Expenditure) × 100
- **Interpretation:** What percent of the budget goes to this category?
- **Example:** `Police_PerExp = 15%` means 15% of budget spent on police
- **Use Case:** Compare spending priorities across cities

#### 2. **Revenue Percentages** (`*_PerRev`)
- **Formula:** (Category Revenue / Total Revenue) × 100
- **Interpretation:** What percent of revenue comes from this source?
- **Example:** `Property_Tax_PerRev = 45%` means 45% of revenue from property tax
- **Use Case:** Compare revenue structure across cities

#### 3. **Per-Capita Rates** (`*_100k`)
- **Formula:** (Count / Population) × 100,000
- **Interpretation:** Rate per 100,000 residents (standard for crime/employment)
- **Example:** `Total_Crime_100k = 2,500` means 2,500 crimes per 100k people
- **Use Case:** Compare crime rates, employment rates across cities

### Key Benefits

✓ **Fair Comparison:** Small and large cities on equal footing  
✓ **Interpretability:** Percentages and rates are intuitive  
✓ **Standard Metrics:** Per-100k is industry standard for crime reporting  
✓ **Preserved Data:** Original columns remain alongside normalized versions

### Concrete Example

Let's examine actual normalized columns alongside their original counterparts:

We'll look at:
1. Hospital expenditure (% of budget)
2. Water utility revenue (% of revenue)
3. Assault rate (per 100k people)

In [ ]:
# Find examples of each normalization type
cols = funcs.search_all(
    ipi_data,
    'name|assault.*munic.*100k|water.*utility.*perrev|total_hospital.*current.*perexp',
    silent=True
)

print(f"Selected {len(cols)} columns demonstrating normalization:")

Name
Total_Hospital_Current_Exp_PerExp
Water_Utility_Revenue_PerRev
Assault- Total - Municipalities_100k


In [ ]:
# Display the normalized features
sample = ipi_data[cols].head()
sample

# INTERPRETATION:
# - *_PerExp: Shows what % of total budget goes to hospitals
# - *_PerRev: Shows what % of revenue comes from water utilities
# - *_100k: Shows assault rate per 100,000 residents (comparable across cities)

,Name,Total_Hospital_Current_Exp_PerExp,Water_Utility_Revenue_PerRev,Assault- Total - Municipalities_100k
0,BOISE CITY,0.0,0.000000,1880.356430
1,KUNA CITY,0.0,27.164502,NaN
2,MERIDIAN CITY,0.0,17.187710,1279.875891
3,BOISE CITY,0.0,0.000000,1693.106451
4,EAGLE CITY,0.0,3.430962,NaN


---

## 4.4 City Classification: `categorize_size()`

Urban and rural communities face different challenges. This function categorizes cities for grouped analysis.

### Classification Rules

Based on U.S. Census Bureau definitions:

| Category | Population Range | Typical Examples |
|----------|------------------|------------------|
| **Rural** | < 2,500 | Small towns, villages |
| **Non-Urban** | 2,500 - 49,999 | Mid-sized towns, small cities |
| **Urban** | ≥ 50,000 | Large cities (Boise, Idaho Falls, etc.) |

### Why Categorize?

Different city sizes have:
- Different revenue sources (small towns rely more on state funding)
- Different expenditure priorities (urban areas need transit, rural don't)
- Different crime patterns (types and rates vary)

### Usage

The `size` column is automatically added by `load.all_data()`. Let's examine it:

In [ ]:
# View city names with their size categories
print("City Classifications:")
ipi_data[['Name', 'Population', 'size']].head(10)

# Summary statistics by city size
print("\n\nCities by Category:")
print(ipi_data.groupby('size')['Name'].nunique())

# Can now compare metrics across city sizes
print("\n\nAverage Crime Rate by City Size:")
print(ipi_data.groupby('size')['Total_Crime_100k'].mean())

,Name,size
0,BOISE CITY,urban
1,KUNA CITY,non-urban
2,MERIDIAN CITY,non-urban
3,BOISE CITY,urban
4,EAGLE CITY,non-urban


## Adjust For Inflation

The money portions money of data we were given was for the year recorded, so we felt it was necessary to adjust the dollar amounts to match current-day amounts. After this function runs, all money columns are adjusted to match October-2019 amounts.

In [11]:
orig_data =  pd.read_excel('Idaho_Municipal_Database_03052019.xlsx', header=1)


In [12]:
print("Un-adjusted")
print(orig_data[['Name','Year4','Total_Expenditure']].head())

print("\n\n After Adjustment\n")
boise = ipi_data.loc[ipi_data['Name'] == "BOISE CITY"].sort_values('Year4')
print(boise[['Name','Year4','Total_Expenditure']].head())

Un-adjusted
         Name  Year4  Total_Expenditure
0  BOISE CITY   1996           116352.0
1  BOISE CITY   1997           142508.0
2  BOISE CITY   1998           147094.0
3  BOISE CITY   1999           148123.0
4  BOISE CITY   2000           157609.0


 After Adjustment

          Name  Year4  Total_Expenditure
0   BOISE CITY   1996      190839.527036
3   BOISE CITY   1997      228497.593570
8   BOISE CITY   1998      232233.451067
12  BOISE CITY   1999      228804.691224
16  BOISE CITY   2000      235540.335157
